# DeepLab v3

## 1. Environment Setup
### 1.0. Check GPU

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


### 1.1. Install Pytorch and torchvision

In [2]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.2. Install scipy

In [3]:
!pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.3. Import

In [4]:
import augmentation, train

import torch
import torchvision

# Ignore Warning
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Preprocessing

In [5]:
train_batch_size = 16
test_batch_size = 1

In [6]:
train_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                             augmentation.RandomResizedCrop((513, 513), scale=(1., 1.), ratio=(0.5, 2.)), 
                                             augmentation.RandomHorizontalFlip(),
                                             augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                           augmentation.Resize(513),
                                           augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.SBDataset(root='./', image_set='train_noval', mode='segmentation', download=False, transforms=train_tf)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_dataset = torchvision.datasets.VOCSegmentation(root='./', year='2012', image_set='val', download=False, transforms=val_tf)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

## 3. Train Model

In [7]:
num_classes = 21
multi_grid = [1, 2, 4]
atrous_rates = [6, 12, 18]
output_stride = 8
scales = [0.5, 0.75, 1, 1.25, 1.5, 1.75]
ignore_index = 255

gpu_id = 4
print_freq = 100
epoch_print = 20

save = False
epochs = 121

lr = 0.007
momentum = 0.9
weight_decay = 0.0005

In [8]:
DeepLab_v3 = train.DeepLab_v3(num_classes=num_classes, multi_grid=multi_grid, atrous_rates=atrous_rates, output_stride=output_stride, 
                              scales=scales, ignore_index=ignore_index, gpu_id=gpu_id, print_freq=print_freq, epoch_print=epoch_print)

In [9]:
DeepLab_v3.train(train_loader, val_loader, save=save, epochs=epochs, lr=lr, momentum=momentum, weight_decay=weight_decay)

Epoch 1 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 100 - Train Loss : 1.048441, Test Loss : 1.238027, Test mIoU : 5.1376


*********************************** Best mIoU Updated ***********************************
Iteration : 200 - Train Loss : 1.090616, Test Loss : 0.987988, Test mIoU : 8.4468


*********************************** Best mIoU Updated ***********************************
Iteration : 300 - Train Loss : 0.833758, Test Loss : 0.710267, Test mIoU : 20.4932

Epoch 21 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 100 - Train Loss : 0.091076, Test Loss : 0.236819, Test mIoU : 71.7617

Iteration : 200 - Train Loss : 0.058038, Test Loss : 0.238690, Test mIoU : 71.4423
Iteration : 300 - Train Loss : 0.088927, Test Loss : 0.239541, Test mIoU : 71.6028
Epoch 41 Started...
Iteration : 100 - Train Loss : 0.065166, Test Loss : 0.246129, Test mIoU : 71